<a href="https://colab.research.google.com/github/buzacott/frontiers_ecohydrology/blob/main/fluxnet2015_ndvi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract NDVI at FLUXNET2015 sites

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import time

BADM (Biological, Ancillary, Disturbance, and Metadata) contain Site General Info, Canopy Height, and Variable Info

Read in BADM data and extract locations

In [ ]:
badm = pd.read_csv('https://raw.githubusercontent.com/buzacott/frontiers_ecohydrology/main/data/fluxnet2015_badm_monthly.csv')

# Lat/lon
location = badm.query('VARIABLE_GROUP == "GRP_LOCATION"') \
               .pivot(index=['SITE_ID', 'GROUP_ID'],
                      columns = 'VARIABLE',
                      values = 'DATAVALUE') \
               .reset_index()
location.drop_duplicates(subset='SITE_ID', inplace=True)

location.head()

Extract key variables

In [ ]:
# Update dtypes, infer_objects() does not work
def update_dtypes(df):
  cols = df.columns
  for c in cols:
      try:
          df[c] = pd.to_numeric(df[c])
      except:
          pass
  return df

location = update_dtypes(location)

## Get NDVI from Google Earth Engine

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

Mapping with Folium

https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb

In [ ]:
import folium
from folium import plugins

# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        # elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
        #     ee_object_new = ee.Image().paint(ee_object, 0, 2)
        #     map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
        #     folium.raster_layers.TileLayer(
        #     tiles = map_id_dict['tile_fetcher'].url_format,
        #     attr = 'Google Earth Engine',
        #     name = name,
        #     overlay = True,
        #     control = True
        # ).add_to(self)
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            map_id_dict = ee_object.getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

Google Earth Engine set up complete

### Extract NDVI

In [ ]:
# Create GEE feature collection of points
features = []
for index, row in location.iterrows():
  features.append(ee.Feature(ee.Geometry.Point([row['LOCATION_LONG'], row['LOCATION_LAT']]),
                             {'SITE_ID': row['SITE_ID']}))
fc = ee.FeatureCollection(features)
# To verify you can run
# fc.getInfo()

In [ ]:
# Buffer points in case they are on the edge of pixel
fc_buf = fc.map(lambda f: f.buffer(50))

In [ ]:
# Load MODIS dataset
dataset = ee.ImageCollection('MODIS/061/MOD13Q1') \
            .filter(ee.Filter.date('2000-01-01', '2015-01-01'));

def applyBitMask(img):
  # Get QA band
  qa = img.select('SummaryQA')
  # 0 or 1 in bits 0-1 indicates good or marginal data
  mask = qa.bitwiseAnd(1 << 1).lte(1)
  return img.updateMask(mask)

# Apply QC mask
dataset = dataset.map(applyBitMask)

ndvi = dataset.select('NDVI') 

In [ ]:
# Create a folium map object
my_map = folium.Map(location=[52.3, 4.8], zoom_start=5, height=500)

# Add custom basemap
basemaps['Google Satellite'].add_to(my_map)

# Add NDVI layer
ndvi_vis = {
  'min': 0.0,
  'max': 9000.0, # NDVI needs to be scaled by 0.0001
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
}
my_map.add_ee_layer(ndvi.first(), ndvi_vis, 'NDVI')
my_map.add_ee_layer(fc, {'color': 'red'}, 'FluxnetSites')
#mapid = fc.getMapId()

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map
display(my_map)

In [ ]:
# Iterate over image collection and get NDVI at fluxnet sites
def get_ndvi_mean(img):
  date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd');

  mean = img.reduceRegions(fc, ee.Reducer.mean(), 250) \
            .map(lambda f: f.set({'date': date}).setGeometry(None))
  return mean

ndvi_ts = ndvi.map(get_ndvi_mean).flatten()

In [ ]:
task = ee.batch.Export.table.toDrive(**{
    'collection': ndvi_ts,
    'description': 'fluxnet2015_modis_ndvi',
    'fileFormat': 'CSV',
    'selectors': ['SITE_ID', 'date', 'mean']
});

In [ ]:
task.start()

while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(30)